In [ ]:
#Instalar e Configurar o H2O
#Importar e Inicializar o H2O

import h2o # type: ignore
from h2o    .grid.grid_search import H2OGridSearch
from h2o.estimators import H2ODeepLearningEstimator

# Iniciar o H2O
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.13+10-LTS-268, mixed mode, sharing)
  Starting server from C:\Users\Avelino\AppData\Local\Programs\Python\Python313\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Avelino\AppData\Local\Temp\tmpnb9nr13y
  JVM stdout: C:\Users\Avelino\AppData\Local\Temp\tmpnb9nr13y\h2o_Avelino_started_from_python.out
  JVM stderr: C:\Users\Avelino\AppData\Local\Temp\tmpnb9nr13y\h2o_Avelino_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_Avelino_2ud83m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.842 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [2]:
#Carregar e Preparar o Dataset

# Carregar o dataset (substitua pelo caminho correto do dataset MNIST)
test_data = h2o.import_file("..\\data\\mnist_test.csv")

# Especificar a coluna de resposta e as variáveis preditoras
response = "label"
predictors = test_data.columns[1:]  # Exclui a primeira coluna que é o rótulo


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
#Configurar o Pré-Processamento dos Dados

# Transformar a coluna de rótulos em fator para classificação
test_data[response] = test_data[response].asfactor()


In [ ]:
# Definir o Modelo de Deep Learning com H2O e Configurar a Grid Search

# Definir hiperparâmetros para a Grid Search
hyper_params = {
    "hidden": [[32, 32], [64, 64], [128, 128]],  # Diferentes combinações de camadas e neurónios
    "rate": [0.01, 0.005, 0.001],                # Diferentes taxas de aprendizagem
    "epochs": [10, 20, 30]                       # Diferentes números de passagens
}

# Criar o modelo de Deep Learning com H2O
dl_model = H2ODeepLearningEstimator(
    activation="RectifierWithDropout",  # Função de ativação com dropout
    input_dropout_ratio=0.2,            # Dropout para a camada de entrada
    hidden_dropout_ratios=[0.5, 0.5],   # Dropout nas camadas ocultas
    loss="CrossEntropy",                # Função de perda para classificação multiclasse
    distribution="multinomial"          # Distribuição para problemas de classificação
)

# Configurar a Grid Search
grid = H2OGridSearch(model=dl_model, hyper_params=hyper_params, grid_id="dl_grid")


In [6]:
#Treinar o Modelo com Grid Search

# Executar a Grid Search
grid.train(x=predictors, y=response, training_frame=test_data, validation_frame=test_data)


deeplearning Grid Build progress: |██████████████████████████████████████████████| (done) 100%


,epochs,hidden,rate,model_ids,logloss
,30.0,"[128, 128]",0.005,dl_grid_model_18,0.0759922
,30.0,"[128, 128]",0.001,dl_grid_model_27,0.0790231
,30.0,"[128, 128]",0.01,dl_grid_model_9,0.0811484
,20.0,"[128, 128]",0.001,dl_grid_model_26,0.1033268
,20.0,"[128, 128]",0.01,dl_grid_model_8,0.1039568
,20.0,"[128, 128]",0.005,dl_grid_model_17,0.1132197
,10.0,"[128, 128]",0.001,dl_grid_model_25,0.1480429
,10.0,"[128, 128]",0.005,dl_grid_model_16,0.1482979
,10.0,"[128, 128]",0.01,dl_grid_model_7,0.1499662
,30.0,"[64, 64]",0.001,dl_grid_model_24,0.1509871


In [7]:
#Analisar os Resultados da Grid Search

# Exibir os resultados da Grid Search e ordenar por acurácia de validação
grid_results = grid.get_grid(sort_by="accuracy", decreasing=True)
print(grid_results)



Hyper-Parameter Search Summary: ordered by decreasing accuracy
     epochs    hidden      rate    model_ids         accuracy
---  --------  ----------  ------  ----------------  ----------
     30.0      [128, 128]  0.005   dl_grid_model_18  0.9755
     30.0      [128, 128]  0.001   dl_grid_model_27  0.9746
     30.0      [128, 128]  0.01    dl_grid_model_9   0.971
     20.0      [128, 128]  0.01    dl_grid_model_8   0.9685
     20.0      [128, 128]  0.001   dl_grid_model_26  0.9658
     20.0      [128, 128]  0.005   dl_grid_model_17  0.9621
     10.0      [128, 128]  0.001   dl_grid_model_25  0.9535
     10.0      [128, 128]  0.005   dl_grid_model_16  0.9524
     10.0      [128, 128]  0.01    dl_grid_model_7   0.9522
     30.0      [64, 64]    0.001   dl_grid_model_24  0.951
---  ---       ---         ---     ---               ---
     10.0      [64, 64]    0.01    dl_grid_model_4   0.9275
     30.0      [32, 32]    0.01    dl_grid_model_3   0.9263
     30.0      [32, 32]    0.005   d

In [8]:
#Avaliar o Melhor Modelo

# Selecionar o melhor modelo
best_model = grid_results.models[0]

# Avaliar o melhor modelo no conjunto de teste
performance = best_model.model_performance(test_data)
print(performance)


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.023365199812083362
RMSE: 0.1528567951125607
LogLoss: 0.07599220946777602
Mean Per-Class Error: 0.024755267154981896
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1     2     3     4    5    6    7     8    9     Error       Rate
---  ----  ----  ----  ---  ---  ---  ----  ---  ----  ----------  ------------
962  0     1     3     0    4    8    1     1    0     0.0183673   18 / 980
0    1124  7     1     0    0    1    1     1    0     0.00969163  11 / 1 135
1    0     1004  0     3    2    7    5     2    8     0.0271318   28 / 1 032
0    0     8     

In [ ]:
# Encerrar a sessão do H2O
h2o.shutdown()

##Resumo
1.  Instalar e inicializar o H2O.
2.  Carregar e preparar os dados: Carregue os dados no formato H2O e transforme a coluna de rótulo em categórica.
3.  Configurar os hiperparâmetros para a Grid Search: Defina as combinações de camadas, taxas de aprendizagem e épocas.
4.  Executar a Grid Search para encontrar a melhor configuração.
5.  Avaliar o melhor modelo selecionado com o conjunto de teste.